In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('/content/drive/MyDrive/undefind_для студентов.csv', sep=';',
                   encoding='utf-8')

In [3]:
# Отображение данных исходного датасета
data.head()

,ID истории болезни,Осн. диаг. при выписке МКБ10 (текст),Заголовок документа,Кол. лаб. показатель,Значение кол. показателя,Ед. изм. кол. показателя,Норма кол. показателя,Флаг нормы кол. показателя,Кач. лаб. показатель,Значение кач. показателя,Норма кач. показателя,Пол,Дата рождения пациента
0,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Плазмоклеточный лейкоз,Общий анализ крови,Лейкоциты (WBC),1.80,10*9/л,4:11,Пониж,NaN,NaN,NaN,Ж,1962-09-22 00:00:00
1,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Плазмоклеточный лейкоз,Общий анализ крови,Эритроциты (RBC),4.18,10*12/л,"3,8:5,7",Норм,NaN,NaN,NaN,Ж,1962-09-22 00:00:00
2,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Плазмоклеточный лейкоз,Общий анализ крови,Гемоглобин (HGB),120.00,г/л,117:160,Норм,NaN,NaN,NaN,Ж,1962-09-22 00:00:00
3,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Плазмоклеточный лейкоз,Общий анализ крови,Гематокрит (HCT),35.10,%,35:47,Норм,NaN,NaN,NaN,Ж,1962-09-22 00:00:00
4,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Плазмоклеточный лейкоз,Общий анализ крови,Средний объем эритроцита (MCV),84.00,фл,80:99,Норм,NaN,NaN,NaN,Ж,1962-09-22 00:00:00


In [ ]:
data.info()

In [ ]:
# Проводим анализ уникальных значений Кол.лаб. показателей.
print(data['Кол. лаб. показатель'].unique())

In [6]:
# Для дальнейшей работы и преобразования датасета создаем копию
GBT_data=data.copy()

In [7]:
# Удаляем неактуальные для данной задачи данные (столбцы)
GBT_data = GBT_data.drop(GBT_data.columns[[1, 7, 8, 9, 10]], axis=1)

In [ ]:
# Отображение данных после преобразования
GBT_data.head()

In [9]:
# Вычисляем возраст пациента поего дате рождения
GBT_data['Дата рождения пациента'] = pd.to_datetime(
    GBT_data['Дата рождения пациента'], errors='coerce')
GBT_data['Возраст'] = (
    pd.Timestamp.now() - GBT_data['Дата рождения пациента']).dt.days // 365

# Удаляем неинформативный столбец "дата рождения пациента"
GBT_data = GBT_data.drop(columns=['Дата рождения пациента'])

# Отображение данных после преобразования
GBT_data.head()

,ID истории болезни,Заголовок документа,Кол. лаб. показатель,Значение кол. показателя,Ед. изм. кол. показателя,Норма кол. показателя,Пол,Возраст
0,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Общий анализ крови,Лейкоциты (WBC),1.80,10*9/л,4:11,Ж,62
1,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Общий анализ крови,Эритроциты (RBC),4.18,10*12/л,"3,8:5,7",Ж,62
2,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Общий анализ крови,Гемоглобин (HGB),120.00,г/л,117:160,Ж,62
3,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Общий анализ крови,Гематокрит (HCT),35.10,%,35:47,Ж,62
4,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Общий анализ крови,Средний объем эритроцита (MCV),84.00,фл,80:99,Ж,62


In [10]:
# Просмотр и анализ уникальных значений столбца "Возраст"
# Минимальный возраст в датасете  - 10 лет. С учетом номрмативных показателей крови у детей старше 10 лет
#   - нецелесообразно создание модели для детского возраста.

print(GBT_data['Возраст'].unique())

[ 62  85  77  66  60  75  72  74  71  47  87  70  51  38  69  48  82  73
  68  76  53  41  63  64  59  80  61  56  84  39  49  55  43  65  81  37
  33 100  44  32  86  50  67  90  79  54  35  42  10]


In [11]:
# Приведение наименований показателей крови к единому (с учетом ранее установленных уникальных значений):
GBT_data['Кол. лаб. показатель'] = GBT_data['Кол. лаб. показатель'].replace(
    ['Эритроциты (RBC)', 'Эритроциты', 'Общее количество эритроцитов (RBC)'],
    'Эритроциты')
GBT_data['Кол. лаб. показатель'] = GBT_data['Кол. лаб. показатель'].replace(
    ['Гемоглобин (HGB)', 'RBC', 'Гемоглобин'], 'Гемоглобин')
GBT_data['Кол. лаб. показатель'] = GBT_data['Кол. лаб. показатель'].replace(
    ['Гематокрит (HCT)', 'Гематокрит', 'HCT'], 'Гематокрит')
GBT_data['Кол. лаб. показатель'] = GBT_data['Кол. лаб. показатель'].replace(
    ['Тромбоциты (PLT)', 'Тромбоциты', 'PLT'], 'Тромбоциты')
GBT_data['Кол. лаб. показатель'] = GBT_data['Кол. лаб. показатель'].replace(
    ['СОЭ Вест.', 'Скорость оседания эритроцитов (СОЭ) по Вестергрену',
     'СОЭ Панч.','СОЭ по Панченкову'], 'СОЭ')
GBT_data['Кол. лаб. показатель'] = GBT_data['Кол. лаб. показатель'].replace(
    ['Лейкоциты (WBC)', 'Лейкоциты', 'WBC',
     'Общее количество лейкоцитов (WBC)'], 'Лейкоциты')


In [ ]:
# Проверяем качество преобразования, просмотр уникальных значений в столбце "Кол. лаб. показатель"

print(GBT_data['Кол. лаб. показатель'].unique())

In [13]:
# Разделение датасета на  датасеты по показателям для создания моделей
RBC_GBT_data = GBT_data[GBT_data['Кол. лаб. показатель'] == 'Эритроциты']
HGB_GBT_data = GBT_data[GBT_data['Кол. лаб. показатель'] == 'Гемоглобин']
HCT_GBT_data = GBT_data[GBT_data['Кол. лаб. показатель'] == 'Гематокрит']
PLT_GBT_data = GBT_data[GBT_data['Кол. лаб. показатель'] == 'Тромбоциты']
ESR_GBT_data = GBT_data[GBT_data['Кол. лаб. показатель'] == 'СОЭ']
WBC_GBT_data = GBT_data[GBT_data['Кол. лаб. показатель'] == 'Лейкоциты']

In [14]:
# Разделение датасетов на датасеты с показателями для мужчин и женщин, где нормы их различаются в зависимости от пола
RBC_GBT_men_data = RBC_GBT_data[RBC_GBT_data['Пол'] == 'М']
RBC_GBT_women_data = RBC_GBT_data[RBC_GBT_data['Пол'] == 'Ж']
HGB_GBT_men_data = HGB_GBT_data[HGB_GBT_data['Пол'] == 'М']
HGB_GBT_women_data = HGB_GBT_data[HGB_GBT_data['Пол'] == 'Ж']
HCT_GBT_men_data = HCT_GBT_data[HCT_GBT_data['Пол'] == 'М']
HCT_GBT_women_data = HCT_GBT_data[HCT_GBT_data['Пол'] == 'Ж']

In [15]:
HCT_GBT_women_data.head()

,ID истории болезни,Заголовок документа,Кол. лаб. показатель,Значение кол. показателя,Ед. изм. кол. показателя,Норма кол. показателя,Пол,Возраст
3,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Общий анализ крови,Гематокрит,35.1,%,35:47,Ж,62
19,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Общий анализ крови,Гематокрит,33.2,%,35:47,Ж,62
49,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Общий анализ крови,Гематокрит,36.3,%,35:47,Ж,62
79,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Общий анализ крови,Гематокрит,30.2,%,35:47,Ж,62
107,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Общий анализ крови,Гематокрит,34.3,%,35:47,Ж,62


In [16]:
# Устанавливаем границы норм и критических значений  по каждому показателю
#Эритроциты:
    # верхняя граница нормы мужчины - 6.1
    # нижняя граница нормы мужчины- 4.3
    # верхняя граница нормы женщины - 5.7
    # нижняя граница нормы женщины- 3.8
RBC_GBT_men_data['Нижняя граница нормы'] = float(4.3)
RBC_GBT_men_data['Верхняя граница нормы'] = float(6.1)
RBC_GBT_women_data['Нижняя граница нормы'] = float(3.8)
RBC_GBT_women_data['Верхняя граница нормы'] = float(5.7)

#Гемоглобин:
    # верхняя граница нормы мужчины - 180
    # нижняя граница нормы мужчины - 132
    # верхняя граница нормы женщины - 160
    # нижняя граница нормы женщины - 117
    # критический верхний показатель -200
    # критический нижний показатель - 60

HGB_GBT_men_data['Нижняя граница нормы'] = float(132)
HGB_GBT_men_data['Верхняя граница нормы'] = float(180)
HGB_GBT_women_data['Нижняя граница нормы'] = float(117)
HGB_GBT_women_data['Верхняя граница нормы'] = float(160)
HGB_GBT_men_data['Критический верхний показатель'] = float(200)
HGB_GBT_men_data['Критический нижний показатель'] = float(60)
HGB_GBT_women_data['Критический верхний показатель'] = float(200)
HGB_GBT_women_data['Критический нижний показатель'] = float(60)

#Тромбоциты:
    # верхняя граница нормы  - 450
    # нижняя граница нормы - 150
    # критический верхний показатель -1000
    # критический нижний показатель - 40
PLT_GBT_data['Нижняя граница нормы'] = float(150)
PLT_GBT_data['Верхняя граница нормы'] = float(450)
PLT_GBT_data['Критический верхний показатель'] = float(1000)
PLT_GBT_data['Критический нижний показатель'] = float(40)

#Лейкоциты:
    # верхняя граница нормы  - 11
    # нижняя граница нормы - 4
    # критический верхний показатель -50
    # критический нижний показатель - 2
WBC_GBT_data['Нижняя граница нормы'] = float(4)
WBC_GBT_data['Верхняя граница нормы'] = float(11)
WBC_GBT_data['Критический верхний показатель'] = float(50)
WBC_GBT_data['Критический нижний показатель'] = float(2)

#Гематокрит:
    # верхняя граница нормы мужчины - 52
    # нижняя граница нормы мужчины- 39
    # верхняя граница нормы женщины - 47
    # нижняя граница нормы женщины- 35
    # критический верхний показатель -60
    # критический нижний показатель - 20
HCT_GBT_men_data['Нижняя граница нормы'] = float(39)
HCT_GBT_men_data['Верхняя граница нормы'] = float(52)
HCT_GBT_women_data['Нижняя граница нормы'] = float(35)
HCT_GBT_women_data['Верхняя граница нормы'] = float(47)
HCT_GBT_men_data['Критический верхний показатель'] = float(60)
HCT_GBT_men_data['Критический нижний показатель'] = float(20)
HCT_GBT_women_data['Критический верхний показатель'] = float(60)
HCT_GBT_women_data['Критический нижний показатель'] = float(20)

#СОЭ:
    # верхняя граница нормы  - 20
    # нижняя граница нормы - 2
ESR_GBT_data['Нижняя граница нормы'] = float(2)
ESR_GBT_data['Верхняя граница нормы'] = float(20)

<ipython-input-16-d7f195e17a46>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RBC_GBT_men_data['Нижняя граница нормы'] = float(4.3)
<ipython-input-16-d7f195e17a46>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  RBC_GBT_men_data['Верхняя граница нормы'] = float(6.1)
<ipython-input-16-d7f195e17a46>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pyda

In [17]:
ESR_GBT_data.head()



,ID истории болезни,Заголовок документа,Кол. лаб. показатель,Значение кол. показателя,Ед. изм. кол. показателя,Норма кол. показателя,Пол,Возраст,Нижняя граница нормы,Верхняя граница нормы
39,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Общий анализ крови,СОЭ,87.0,мм/час,2:20,Ж,62,2.0,20.0
69,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Общий анализ крови,СОЭ,88.0,мм/час,2:20,Ж,62,2.0,20.0
98,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Общий анализ крови,СОЭ,112.0,мм/час,2:20,Ж,62,2.0,20.0
126,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Общий анализ крови,СОЭ,52.0,мм/час,2:20,Ж,62,2.0,20.0
196,24612d4d-c466-11ec-ab54-0050568844e6,Общий анализ крови,СОЭ,44.0,мм/час,2:20,Ж,85,2.0,20.0


In [32]:
# Установим Категории значения показателя:
    # 1 - критическое снижение показателя
    # 2 - некритическое снижение показателя
    # 3 - вариант нормы
    # 4 - некритическое повышение показателя
    # 5 - критическое повышение показателя

# Функция для определения значения столбца "Категория значения показателя"  при наличии установленных критических значений
def assign_category(row):
    if row['Значение кол. показателя'] <= row['Критический нижний показатель']:
        return 1
    elif row['Критический нижний показатель'] \
        < row['Значение кол. показателя'] <  row['Нижняя граница нормы']:
        return 2
    elif row['Нижняя граница нормы'] \
        <= row['Значение кол. показателя'] <= row['Верхняя граница нормы']:
        return 3
    elif row['Верхняя граница нормы'] \
        < row['Значение кол. показателя'] < row['Критический верхний показатель']:
        return 4
    else:
        return 5

# Добавляем столбец "Категория значения показателя" в датафреймы и заполняем значениями с помощью функции assign_category
HGB_GBT_men_data['Категория значения показателя'] = HGB_GBT_men_data.apply(
                                                    assign_category, axis=1)
HGB_GBT_women_data['Категория значения показателя'] = HGB_GBT_women_data.apply(
                                                    assign_category, axis=1)
PLT_GBT_data['Категория значения показателя'] = PLT_GBT_data.apply(
                                                assign_category, axis=1)
WBC_GBT_data['Категория значения показателя'] = WBC_GBT_data.apply(
                                                assign_category, axis=1)
HCT_GBT_men_data['Категория значения показателя'] = HCT_GBT_men_data.apply(
                                                    assign_category, axis=1)
HCT_GBT_women_data['Категория значения показателя'] = HCT_GBT_women_data.apply(
                                                      assign_category, axis=1)
# Функция для определения значения столбца "Категория значения показателя"  при отсутствии установленных критических значений
def fill_column(row):
    if row['Значение кол. показателя'] < row['Нижняя граница нормы']:
        return 2
    elif row['Нижняя граница нормы'] <= row['Значение кол. показателя'] \
        <= row['Верхняя граница нормы']:
        return 3
    else:
        return 4

# Добавляем столбец "Категория значения показателя" в датафреймы и заполняем значениями с помощью функции
RBC_GBT_men_data['Категория значения показателя'] = RBC_GBT_men_data.apply(
                                                    fill_column, axis=1)
RBC_GBT_women_data['Категория значения показателя'] = RBC_GBT_women_data.apply(
                                                      fill_column, axis=1)
ESR_GBT_data['Категория значения показателя'] = ESR_GBT_data.apply(
                                                fill_column, axis=1)


<ipython-input-32-9ece4eab4339>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HGB_GBT_men_data['Категория значения показателя'] = HGB_GBT_men_data.apply(
<ipython-input-32-9ece4eab4339>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  HGB_GBT_women_data['Категория значения показателя'] = HGB_GBT_women_data.apply(
<ipython-input-32-9ece4eab4339>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

In [33]:
RBC_GBT_women_data.head()

,ID истории болезни,Заголовок документа,Кол. лаб. показатель,Значение кол. показателя,Ед. изм. кол. показателя,Норма кол. показателя,Пол,Возраст,Нижняя граница нормы,Верхняя граница нормы,Категория значения показателя
1,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Общий анализ крови,Эритроциты,4.18,10*12/л,"3,8:5,7",Ж,62,3.8,5.7,3
17,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Общий анализ крови,Эритроциты,3.64,10*12/л,"3,8:5,7",Ж,62,3.8,5.7,2
47,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Общий анализ крови,Эритроциты,4.00,10*12/л,"3,8:5,7",Ж,62,3.8,5.7,3
77,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Общий анализ крови,Эритроциты,3.37,10*12/л,"3,8:5,7",Ж,62,3.8,5.7,2
105,2e1d0b3f-488a-11ed-ab5a-0050568844e6,Общий анализ крови,Эритроциты,3.92,10*12/л,"3,8:5,7",Ж,62,3.8,5.7,3


**Модель линейной регресии для показателя "СОЭ"**

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

X = ESR_GBT_data[['Значение кол. показателя']]
y = ESR_GBT_data['Категория значения показателя']

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

In [36]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [37]:
y_pred = model.predict(X_test)

In [38]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность модели: {accuracy}')

Точность модели: 1.0


**Модель линейной регрессии для показателя "Лейкоциты"**

In [39]:
X = WBC_GBT_data[['Значение кол. показателя']]
y = WBC_GBT_data['Категория значения показателя']

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

In [41]:
model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [42]:
y_pred = model.predict(X_test)

In [43]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность модели: {accuracy}')

Точность модели: 0.9938271604938271


**Модель линейной регресии для показателя "Тромбоциты"**

In [44]:
X= PLT_GBT_data[['Значение кол. показателя']]
y = PLT_GBT_data['Категория значения показателя']

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

In [46]:
model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [47]:
y_pred = model.predict(X_test)

In [48]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность модели: {accuracy}')

Точность модели: 1.0


**Модель линейной регрессии для показателя "Гемоглобин. Мужчины"**

In [49]:
X= HGB_GBT_men_data[['Значение кол. показателя']]
y = HGB_GBT_men_data['Категория значения показателя']

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

In [51]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [52]:
y_pred = model.predict(X_test)

In [53]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность модели: {accuracy}')

Точность модели: 1.0


**Модель линейной регрессии для показателя "Гемоглобин. Женщины"**

In [54]:
X= HGB_GBT_women_data[['Значение кол. показателя']]
y = HGB_GBT_women_data['Категория значения показателя']

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

In [56]:
model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [57]:
y_pred = model.predict(X_test)

In [58]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность модели: {accuracy}')

Точность модели: 1.0


**Модель линейной регрессии для показателя "Эритроциты. Мужчины"**

In [59]:
X= RBC_GBT_men_data[['Значение кол. показателя']]
y = RBC_GBT_men_data['Категория значения показателя']

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

In [61]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [62]:
y_pred = model.predict(X_test)

In [63]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность модели: {accuracy}')

Точность модели: 0.9818181818181818


**Модель линейной регрессии для показателя "Эритроциты. Женщины"**

In [64]:
X= RBC_GBT_women_data[['Значение кол. показателя']]
y = RBC_GBT_women_data['Категория значения показателя']

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

In [66]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [67]:
y_pred = model.predict(X_test)

In [68]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность модели: {accuracy}')

Точность модели: 1.0


**Модель линейной регрессии для показателя "Гематокрит. Мужчины"**

In [69]:
X= RBC_GBT_women_data[['Значение кол. показателя']]
y = RBC_GBT_women_data['Категория значения показателя']

In [70]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

In [71]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [72]:
y_pred = model.predict(X_test)

In [73]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность модели: {accuracy}')

Точность модели: 1.0


**Модель линейной регрессии для показателя "Гематокрит. Женщины"**

In [74]:
HCT_GBT_men_data
X= RBC_GBT_women_data[['Значение кол. показателя']]
y = RBC_GBT_women_data['Категория значения показателя']

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

In [76]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [77]:
y_pred = model.predict(X_test)

In [78]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность модели: {accuracy}')

Точность модели: 1.0
